# 의상 코디 피드백 평가하기
---

## 실험 환경
- 이 노트북은 SageMaker Studio Code Editor 및 커널 base (Python 3.10.13) 에서 테스트 되었습니다.
- 실행 환경에 설치된 Python Package 참고 하세요. --> [requirements.txt](../requirements.txt)

## 참고 자료
- [Building with Anthropic’s Claude 3 on Amazon Bedrock and LangChain](https://medium.com/@dminhk/building-with-anthropics-claude-3-on-amazon-bedrock-and-langchain-%EF%B8%8F-2b842f9c0ca8)
- [Amazon Bedrock 기반 Amorepacific 리뷰 요약 서비스 평가 방법 구현하기](langchain_core.runnables.base.RunnableSequence)
- [Amazon Bedrock model IDs](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html)

---

# 0. 선수 노트북
- 아래의 노트북을 먼저 실행하셔야 합니다.  --> [01_matching_codi_product.ipynb](01_matching_codi_product.ipynb)


# 1. 환경 셋업

가장 처음 실행시에는 아래를 install_needed == True 로 실행 해주세요

In [1]:
import sys

# install_needed = True
install_needed = False

if install_needed:
    !{sys.executable} -m pip install -q langchain==0.1.16
    !{sys.executable} -m pip install -q langchain-community==0.0.34
    !{sys.executable} -m pip install -q langchain-aws==0.1.2
    !{sys.executable} -m pip install -q boto3==1.34.87    

In [2]:
! pip list | grep -E "langchain|boto3"

boto3                                 1.34.87
langchain                             0.1.16
langchain-aws                         0.1.2
langchain-community                   0.0.34
langchain-core                        0.1.46
langchain-text-splitters              0.0.1


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys, os

def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)

module_path = ".."
add_python_path(module_path)

python path: /home/sagemaker-user/aws-ai-ml-workshop-kr-1/genai/aws-gen-ai-kr/20_applications/05_image_styling_recommendation_with_prompt_engineering/evaluation is added
sys.path:  ['/home/sagemaker-user/aws-ai-ml-workshop-kr-1/genai/aws-gen-ai-kr/20_applications/05_image_styling_recommendation_with_prompt_engineering/evaluation/notebook', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages', '/home/sagemaker-user/aws-ai-ml-workshop-kr-1/genai/aws-gen-ai-kr/20_applications/05_image_styling_recommendation_with_prompt_engineering/evaluation']


# 2. Ground Truth (Label) 생성하기

## 두개의 모델에서 선택한 코디 이유 가져오기
- 이전 노트북에서 저장한 두개의 모델의 이유를 가져 옵니다

In [5]:
from eval_utils.langchain_bedrock import BedrockLangChain
from eval_utils.util import (
                        print_ww
                        # invoke_model_text2text,
                        
)


In [6]:
%store -r select_reason_sonnet
%store -r select_reason_haiku

try:
    print("select_reason_sonnet:")
    print_ww(select_reason_sonnet)
    print("select_reason_haiku: ")
    print_ww(select_reason_haiku)
except:
    print("Run 01_matching_codi_product.ipynb")




select_reason_sonnet:
이 제품은 데님 재킷과 잘 어울리는 심플하고 베이직한 화이트 반팔 티셔츠입니다. 티셔츠의 베이직한 디자인과 데님 재킷의 캐주얼한 스타일이 잘 매치되어 전체적으로 편안하고 자연스러운
데일리 룩을 연출할 수 있습니다. 또한 티셔츠의 화이트 색상은 블루 데님 재킷과 대비를 이루어 밝고 산뜻한 느낌을 줍니다.
select_reason_haiku: 
데님 재킷과 가장 잘 어울리는 아이템은 캐주얼하고 편안한 데일리 룩인 반팔 티셔츠입니다. 데님 재킷과 티셔츠의 조합은 전형적인 캐주얼 스타일을 연출할 수 있으며, 특히 이 반팔
티셔츠는 심플하고 베이직한 디자인으로 데님 재킷과 잘 어울립니다. 또한 청바지와 함께 착용하면 전체적으로 편안하고 자연스러운 분위기를 연출할 수 있습니다.


## Ground Truth 생성을 위한 System, User Prompt 확인하기
- Ground Truth 를 사람이 작성하는 것을 권장하나, 너무 많은 노력이 들어 갈 수 있어서 여기서는 아래와 같이 가정하여 작성합니다.
    - Claude3 Sonnet 모델이 어울리다가 생각한 "이유" 를 Claude3 Sonnet 모델에 Re-Write 한 것을 Ground Truth 가정 합니다.
- 아래의 System Prompt & User Prompt 는 미리 Claude3 Sonnet 모델을 이용해서 작성한 것을 가져옵니다.

In [7]:
import boto3
from eval_utils.prompt import FashionPrompt

FP = FashionPrompt()
system_prompt = FP.get_rewrite_system_prompt()
user_prompt = FP.get_rewrite_user_prompt()

print("## system_prompt")
print_ww(system_prompt)
print("## user_prompt")
print_ww(user_prompt)


## system_prompt
The task is to rewrite a given sentence in a different way while preserving its original
meaning.Your role is to take a sentence provided by the user and rephrase it using different words
or sentence structures, without altering the core meaning or message conveyed in the original
sentence.

Instructions:
1. Read the sentence carefully and ensure you understand its intended meaning.
2. Identify the key components of the sentence, such as the subject, verb, object, and any modifiers
or additional information.
3. Think of alternative ways to express the same idea using different vocabulary, sentence
structures, or phrasing.
4. Ensure that your rewritten sentence maintains the same essential meaning as the original, without
introducing any new information or altering the original intent.
5. Pay attention to grammar, punctuation, and overall coherence to ensure your rewritten sentence is
well-formed and easy to understand.
6. If the original sentence contains idioms, metap

## Ground Truth 를 Sonnet 모델을 이용하여 Rw-Write 하여 생성

In [8]:

region = "us-west-2"
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

client = boto3.client(service_name="bedrock-runtime", region_name=region)
model_kwargs =  { 
    "max_tokens": 2048,
    "temperature": 1.0,
    "top_k": 250,
    "top_p": 1,
}                             

BL = BedrockLangChain(bedrock_runtime=client)
response = BL.invoke_rewrite_langchain(model_id = model_id, 
                             model_kwargs = model_kwargs, 
                             user_prompt = user_prompt, 
                             system_prompt = system_prompt, 
                             coordination_review = select_reason_sonnet,
                             verbose=True)

import json
data_dict = json.loads(response)
rewrite_original_coordination_review = data_dict['rewrite_original_coordination_review']
print("## Re-Write coordination_review ")
print_ww(rewrite_original_coordination_review)

messages: 
 [('system', 'The task is to rewrite a given sentence in a different way while preserving its original meaning.Your role is to take a sentence provided by the user and rephrase it using different words or sentence structures, without altering the core meaning or message conveyed in the original sentence.\n\nInstructions:\n1. Read the sentence carefully and ensure you understand its intended meaning.\n2. Identify the key components of the sentence, such as the subject, verb, object, and any modifiers or additional information.\n3. Think of alternative ways to express the same idea using different vocabulary, sentence structures, or phrasing.\n4. Ensure that your rewritten sentence maintains the same essential meaning as the original, without introducing any new information or altering the original intent.\n5. Pay attention to grammar, punctuation, and overall coherence to ensure your rewritten sentence is well-formed and easy to understand.\n6. If the original sentence contai

# 4. Evaluation 기준 및 방법을 생성하는 프롬프트 만들기
- "Evaluation 기준 및 방법" 을 여러가지 솔루션으로 구현할 수 있습니다. 여기서는 Claude3 Sonnet 에게 이런한 기준을 작성하게 했습니다.
    - 아래는 이러한 기준을 작성할 수 있다는 예시 입니다. 
    - 실제로 아래 프롬프트를 사용하여 생성을 받았고, 이후에 약간의 편집을 통해서 ../eval_utils/prompt.py 에 저장을 했습니다. 
    - 여기 셀의 실행은 이렇게 할 수 있다는 것만을 보여 드립니다.


## Evaluation 기준 및 방법을 생성하는 프롬프트



In [9]:
from eval_utils.prompt import FashionPrompt

model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

model_kwargs =  { 
    "max_tokens": 2048,
    "temperature": 0.0,
    "top_k": 250,
    "top_p": 1,
}                             


FP = FashionPrompt()
system_prompt = FP.get_create_criteria_system_prompt()
user_prompt = FP.get_create_criteria_user_prompt()

guide="이후에 당신은 의상 전문가의 의상 착용에 대한  의견과 AI 가 제공한 의상 착용의  두가지 의견이 주어질 겁니다.  \
이 두개의 의견이 얼마나 연관성이 있는지를 1점, 2점, 3점, 4점, 5점 으로서  스코어의 기준을 <criterira></criterira>  형식으로 작성하시고, \
두개의 의견이 제공이 되었을때에 평가를 하는 단계를 <steps></steps> 형식으로 작성 하세요. "


BL = BedrockLangChain(bedrock_runtime=client)
BL.invoke_creating_criteria_langchain(model_id = model_id, 
                             model_kwargs = model_kwargs, 
                             user_prompt = user_prompt, 
                             system_prompt = system_prompt, 
                             guide = guide,
                             verbose=True)


messages: 
 [('system', 'You are a prompt engineering expert.'), ('human', '먼저 당신의 역할과 작업을 XML Tag 없이 기술하세요, 이후에 아래의 <guide> 에 맟주어서 프롬프트를 영어로 작성해주세요. \n<guide>{guide}</guide>')]
prompt: 

input_variables=['guide']
messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a
prompt engineering expert.')),
HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['guide'], template='먼저 당신의 역할과 작업을
XML Tag 없이 기술하세요, 이후에 아래의 <guide> 에 맟주어서 프롬프트를 영어로 작성해주세요. \n<guide>{guide}</guide>'))]
## Created Prompt:



As a prompt engineering expert, my role is to provide clear and effective prompts to guide AI language models in generating relevant and coherent responses. I aim to structure prompts in a way that aligns the AI's output with the desired task or goal.

<guide>
You will be provided with two opinions: one from a fashion expert regarding clothing choices, and another from an AI system offering recommendations on clothing choices. Your task is to evaluate the relevance and coherence between these two opinions by assigning a score from 1 to 5.

<criteria>
1 - The two opinions are completely unrelated and have no coherence.
2 - The opinions share minimal relevance or coherence.
3 - The opinions have some degree of relevance and coherence, but with notable differences.
4 - The opinions are largely relevant and coherent, with only minor differences.
5 - The two opinions are highly relevant and coherent, aligning closely with each other.
</criteria>

<steps>
1. Carefully read and understand the

# 5. Ground Truth 와 AI Review 평가하기

## Claude3 Sonnet 이 제공한 이유 평가하기
- ground_truth 와 Sonnet 이 작성한 이유를 비교해서 "관련성" 을 평가 합니다.

In [10]:
system_prompt = FP.get_fashion_evaluation_system_prompt()
user_prompt = FP.get_fashion_evaluation_user_prompt()


ground_truth = rewrite_original_coordination_review

select_reason_haiku

# human_message = "원피스 아이템에 트랜치코드를 걸치면 세련되고 캐주얼한 스타일을 연출할 수 있습니다. 베이지색 트랜치코트는 네이비 원피스의 색상과 잘 어울리며, 면 소재는 편안한 착용감을 제공합니다. 또한 버킷백과도 스타일리시하게 매치할 수 있어 전체적으로 조화로운 코디가 가능합니다"                  
# AI_message = "데님 팬츠와 잘 어울리는 상의는 기본적인 셔츠 디자인이 적절합니다. 스트라이프 셔츠는 심플하면서도 포인트 디테일(노란색 자)이 있어 팬츠와 잘 매치될 것 같습니다. 또한 면 소재라 데님 팬츠와 소재감도 비슷하여 코디하기 좋을 것 같습니다"                  


human_message = ground_truth
AI_message = select_reason_sonnet


BL = BedrockLangChain(bedrock_runtime=client)
BL.invoke_evaluating_fashion_review_langchain(model_id = model_id, 
                             model_kwargs = model_kwargs, 
                             user_prompt = user_prompt, 
                             system_prompt = system_prompt, 
                             AI_message = AI_message,
                             human_message = human_message,
                             verbose=False)


<evaluation>
{
"human_view": "이 제품은 평범한 화이트 반팔 티셔츠지만 데님 재킷과 함께 입으면 캐주얼하면서도 자연스러운 스타일을 연출할 수 있습니다. 티셔츠의 단정한 디자인과 데님 재킷의 편안한 느낌이 잘 어우러져 일상생활에서 부담 없이 입을 수 있는 옷차림을 완성합니다. 또한 화이트와 블루 데님의 색상 대비가 전체적인 룩에 산뜻함과 밝은 분위기를 더해줍니다.",
"AI_view": "이 제품은 데님 재킷과 잘 어울리는 심플하고 베이직한 화이트 반팔 티셔츠입니다. 티셔츠의 베이직한 디자인과 데님 재킷의 캐주얼한 스타일이 잘 매치되어 전체적으로 편안하고 자연스러운 데일리 룩을 연출할 수 있습니다. 또한 티셔츠의 화이트 색상은 블루 데님 재킷과 대비를 이루어 밝고 산뜻한 느낌을 줍니다.",
"score": 4,
"reason": "인간과 AI의 의견이 대체로 일치하며 티셔츠와 데님 재킷의 조화로운 매치에 대해 유사한 관점을 제시합니다. 다만 AI 의견에서는 티셔츠 디자인을 '베이직'하다고 표현한 반면 인간 의견에서는 '단정한'이라고 표현한 점에서 약간의 차이가 있습니다."
}
</evaluation>

## Claude3 Haiku 이 제공한 이유 평가하기
- ground_truth 와 Haiku 가 작성한 이유를 비교해서 "관련성" 을 평가 합니다.

In [11]:
ground_truth = rewrite_original_coordination_review

human_message = ground_truth
AI_message = select_reason_haiku


BL = BedrockLangChain(bedrock_runtime=client)
BL.invoke_evaluating_fashion_review_langchain(model_id = model_id, 
                             model_kwargs = model_kwargs, 
                             user_prompt = user_prompt, 
                             system_prompt = system_prompt, 
                             AI_message = AI_message,
                             human_message = human_message,
                             verbose=False)


{
"human_view": "이 제품은 평범한 화이트 반팔 티셔츠지만 데님 재킷과 함께 입으면 캐주얼하면서도 자연스러운 스타일을 연출할 수 있습니다. 티셔츠의 단정한 디자인과 데님 재킷의 편안한 느낌이 잘 어우러져 일상생활에서 부담 없이 입을 수 있는 옷차림을 완성합니다. 또한 화이트와 블루 데님의 색상 대비가 전체적인 룩에 산뜻함과 밝은 분위기를 더해줍니다.",
"AI_view": "데님 재킷과 가장 잘 어울리는 아이템은 캐주얼하고 편안한 데일리 룩인 반팔 티셔츠입니다. 데님 재킷과 티셔츠의 조합은 전형적인 캐주얼 스타일을 연출할 수 있으며, 특히 이 반팔 티셔츠는 심플하고 베이직한 디자인으로 데님 재킷과 잘 어울립니다. 또한 청바지와 함께 착용하면 전체적으로 편안하고 자연스러운 분위기를 연출할 수 있습니다.",
"score": 4,
"reason": "두 의견은 대체로 일치하며, 데님 재킷과 반팔 티셔츠의 조합이 캐주얼하고 자연스러운 스타일을 연출한다는 점에서 유사한 관점을 제시합니다. 다만 AI 의견은 청바지와의 조합에 대해 언급한 반면, 인간 의견은 색상 대비에 대해 더 자세히 설명하고 있습니다."
}

## 실험으로 관련성이 적은 리뷰 제공
- Sonnet, Haiku 가 비슷한 스코어를 (예: 4점) 을 주어서, AI_review 가 다른 것을 입력으로 하여, 스코어를 비교 합니다.

In [12]:
ground_truth = rewrite_original_coordination_review


human_message = ground_truth
AI_message = "원피스 아이템에 트랜치코드를 걸치면 세련되고 캐주얼한 스타일을 연출할 수 있습니다. 베이지색 트랜치코트는 네이비 원피스의 색상과 잘 어울리며, 면 소재는 편안한 착용감을 제공합니다. 또한 버킷백과도 스타일리시하게 매치할 수 있어 전체적으로 조화로운 코디가 가능합니다"                  


BL = BedrockLangChain(bedrock_runtime=client)
BL.invoke_evaluating_fashion_review_langchain(model_id = model_id, 
                             model_kwargs = model_kwargs, 
                             user_prompt = user_prompt, 
                             system_prompt = system_prompt, 
                             AI_message = AI_message,
                             human_message = human_message,
                             verbose=False)


{
"human_view": "이 제품은 평범한 화이트 반팔 티셔츠지만 데님 재킷과 함께 입으면 캐주얼하면서도 자연스러운 스타일을 연출할 수 있습니다. 티셔츠의 단정한 디자인과 데님 재킷의 편안한 느낌이 잘 어우러져 일상생활에서 부담 없이 입을 수 있는 옷차림을 완성합니다. 또한 화이트와 블루 데님의 색상 대비가 전체적인 룩에 산뜻함과 밝은 분위기를 더해줍니다.",
"AI_view": "원피스 아이템에 트랜치코드를 걸치면 세련되고 캐주얼한 스타일을 연출할 수 있습니다. 베이지색 트랜치코트는 네이비 원피스의 색상과 잘 어울리며, 면 소재는 편안한 착용감을 제공합니다. 또한 버킷백과도 스타일리시하게 매치할 수 있어 전체적으로 조화로운 코디가 가능합니다",
"score": 2,
"reason": "두 의견은 옷차림에 대한 일반적인 조언을 제공하지만 구체적인 아이템과 스타일링 방식이 다릅니다. 인간 의견은 티셔츠와 데님 재킷 조합을 다루는 반면, AI 의견은 원피스와 트렌치코트 조합을 다룹니다."
}